In [2]:
!pip install -q tflite-model-maker

ERROR: Cannot install tflite-model-maker==0.1.2, tflite-model-maker==0.2.0, tflite-model-maker==0.2.1, tflite-model-maker==0.2.2, tflite-model-maker==0.2.3, tflite-model-maker==0.2.4, tflite-model-maker==0.2.5, tflite-model-maker==0.3.3, tflite-model-maker==0.3.4, tflite-model-maker==0.4.0, tflite-model-maker==0.4.1, tflite-model-maker==0.4.2 and tflite-model-maker==0.4.3 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [ ]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

In [ ]:
print("load start...")

train_data = DataLoader.from_folder(image_path)

val_data= DataLoader.from_folder(image_path)

print("load end")

In [ ]:
print("split start...")

validation_data, test_data = val_data.split(0.5)

print("split end")

In [ ]:
print("train start...")
model = image_classifier.create(train_data, validation_data=validation_data)
print("train end")

In [ ]:
model.summary()

In [ ]:
print("eval start...")
loss, accuracy = model.evaluate(test_data)
print("eval end")

In [ ]:
print("export start...")
model.export(export_dir='.')
print("export end")

In [ ]:
# A helper function that returns 'red'/'black' depending on if its two input
# parameter matches or not.
def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

# Then plot 100 test images and their predicted labels.
# If a prediction result is different from the label provided label in "test"
# dataset, we will highlight it in red color.
plt.figure(figsize=(20, 20))
predicts = model.predict_top_k(test_data)
for i, (image, label) in enumerate(test_data.gen_dataset().unbatch().take(100)):
  ax = plt.subplot(10, 10, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)

  predict_label = predicts[i][0][0]
  color = get_label_color(predict_label,
                          test_data.index_to_label[label.numpy()])
  ax.xaxis.label.set_color(color)
  plt.xlabel('Predicted: %s' % predict_label)
plt.show()